# Traffic-based Route Guidance Solution

## Data Preprocessing & Analyzing

In [207]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

1. Load the VicRoads Boroondara dataset (.csv)

2. Clean and preprocess:
    * Convert timestamps

    * Handle missing values

    * Normalize/scale traffic flow values

3. Reshape for time-series forecasting (e.g., sequences of past 1-2 hours to predict next 15-min slot)

### Import the Dataset for analyze

In [5]:
df = pd.read_csv('../data/raw/Scats Data October 2006.csv', skiprows=1)
df

,SCATS Number,Location,CD_MELWAY,NB_LATITUDE,NB_LONGITUDE,HF VicRoads Internal,VR Internal Stat,VR Internal Loc,NB_TYPE_SURVEY,Date,...,V89,V90,V91,V92,V93,V94,V95,Unnamed: 106,Unnamed: 107,Unnamed: 108
0,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,1/10/2006,...,66,81,50,59,47,29,34,NaN,NaN,NaN
1,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,2/10/2006,...,114,80,60,62,48,44,26,NaN,NaN,NaN
2,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,3/10/2006,...,86,93,90,73,57,29,40,NaN,NaN,NaN
3,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,4/10/2006,...,101,113,90,78,66,52,44,NaN,NaN,NaN
4,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,5/10/2006,...,113,99,91,61,55,49,36,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,4821,VICTORIA_ST W OF BURNLEY_ST,002HF02,-37.81296,145.00830,6673,1513,7,1,27/10/2006,...,103,122,124,117,99,108,88,NaN,NaN,NaN
4188,4821,VICTORIA_ST W OF BURNLEY_ST,002HF02,-37.81296,145.00830,6673,1513,7,1,28/10/2006,...,105,105,112,82,97,106,107,NaN,NaN,NaN
4189,4821,VICTORIA_ST W OF BURNLEY_ST,002HF02,-37.81296,145.00830,6673,1513,7,1,29/10/2006,...,76,66,64,77,60,49,45,NaN,NaN,NaN
4190,4821,VICTORIA_ST W OF BURNLEY_ST,002HF02,-37.81296,145.00830,6673,1513,7,1,30/10/2006,...,80,74,48,67,62,50,62,NaN,NaN,NaN


In [ ]:
# Create time mapping for V00–V95 (15-minute intervals)
time_labels = [f"{h:02}:{m:02}:00" for h in range(24) for m in range(0, 60, 15)]
v_columns = [f"V{str(i).zfill(2)}" for i in range(96)]
time_map = dict(zip(v_columns, time_labels))

# Melt the V columns into long format
df_melted = df.melt(
    id_vars=["SCATS Number", "Location", "Date"],
    value_vars=v_columns,
    var_name="time_code",
    value_name="volume"
)

# Map time codes to actual time strings
df_melted['timestamp'] = df_melted['time_code'].map(time_map)

# Combine Date and Time into a full timestamp
# df_melted['timestamp'] = pd.to_datetime(df_melted['Date'] + ' ' + df_melted['time_str'], dayfirst=True)

# Rename columns for clarity
df_melted = df_melted.rename(columns={
    "SCATS Number": "site_id",
    "Location": "location",
    "Date": "date"
})

# Select and reorder important columns
df_traffic = df_melted[["site_id", "location", "date", "timestamp", "volume"]]

# # Load site coordinate data
# df_coords = pd.read_csv("../data/raw/Traffic_Count_Locations_with_LONG_LAT.csv")
# df_coords = df_coords.rename(columns={
#     "TFM_ID": "site_id",
#     "X": "longitude",
#     "Y": "latitude"
# })

# Merge coordinates into traffic data
# df_combined = df_traffic.merge(df_coords[["site_id", "latitude", "longitude"]], on="site_id", how="left")

df_combined = df_traffic.sort_values(by=['site_id','date', 'timestamp'])
# Print data after clean and preprocess
print(df_combined)
x=0
# Cluster by site_id
for i in df_combined.site_id.unique():
    x+=1    
    
print('SCATS Count: ',x)

for i in df_combined.site_id.unique():
    print('Site ID: ', i)
    for route in :
     

        site_id                         location       date timestamp  volume
0           970  WARRIGAL_RD N of HIGH STREET_RD  1/10/2006  00:00:00      86
31          970  HIGH STREET_RD E of WARRIGAL_RD  1/10/2006  00:00:00      37
62          970  WARRIGAL_RD S of HIGH STREET_RD  1/10/2006  00:00:00      92
93          970  HIGH STREET_RD W of WARRIGAL_RD  1/10/2006  00:00:00      47
4192        970  WARRIGAL_RD N of HIGH STREET_RD  1/10/2006  00:15:00      83
...         ...                              ...        ...       ...     ...
398217     4821      VICTORIA_ST W OF BURNLEY_ST  9/10/2006  23:30:00      49
402316     4821       WALMER_ST N OF VICTORIA_ST  9/10/2006  23:45:00       0
402347     4821      VICTORIA_ST E OF BURNLEY_ST  9/10/2006  23:45:00      19
402378     4821      BURNLEY_ST S OF VICTORIA_ST  9/10/2006  23:45:00      18
402409     4821      VICTORIA_ST W OF BURNLEY_ST  9/10/2006  23:45:00      37

[402432 rows x 5 columns]
SCATS Count:  40
Site ID:  970
Site I

In [7]:
data = df_combined.values

#View the data before processing
print(data)

[[970 'WARRIGAL_RD N of HIGH STREET_RD' '1/10/2006' '00:00:00' 86]
 [970 'WARRIGAL_RD N of HIGH STREET_RD' '2/10/2006' '00:00:00' 32]
 [970 'WARRIGAL_RD N of HIGH STREET_RD' '3/10/2006' '00:00:00' 26]
 ...
 [4821 'VICTORIA_ST W OF BURNLEY_ST' '29/10/2006' '23:45:00' 45]
 [4821 'VICTORIA_ST W OF BURNLEY_ST' '30/10/2006' '23:45:00' 62]
 [4821 'VICTORIA_ST W OF BURNLEY_ST' '31/10/2006' '23:45:00' 54]]


### Data processing for ML

#### Create Time Series Sequences

In [ ]:
def create_sequences(data, seq_length):
    x, y = [], []
    for i in range(len(data) - seq_length):
        x.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(x), np.array(y)

SEQ_LEN = 30 # use last 24 hours to predict next
x, y = create_sequences(data, SEQ_LEN)
# Locations = df_traffic['location'].unique()
# print(len(Locations))

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [9]:
i = 1
print("x : ", x[i])
print("y : ", y[i])

x :  [[970 'WARRIGAL_RD N of HIGH STREET_RD' '2/10/2006' '00:00:00' 32]
 [970 'WARRIGAL_RD N of HIGH STREET_RD' '3/10/2006' '00:00:00' 26]
 [970 'WARRIGAL_RD N of HIGH STREET_RD' '4/10/2006' '00:00:00' 32]
 [970 'WARRIGAL_RD N of HIGH STREET_RD' '5/10/2006' '00:00:00' 40]
 [970 'WARRIGAL_RD N of HIGH STREET_RD' '6/10/2006' '00:00:00' 36]
 [970 'WARRIGAL_RD N of HIGH STREET_RD' '7/10/2006' '00:00:00' 62]
 [970 'WARRIGAL_RD N of HIGH STREET_RD' '8/10/2006' '00:00:00' 116]
 [970 'WARRIGAL_RD N of HIGH STREET_RD' '9/10/2006' '00:00:00' 23]
 [970 'WARRIGAL_RD N of HIGH STREET_RD' '10/10/2006' '00:00:00' 27]
 [970 'WARRIGAL_RD N of HIGH STREET_RD' '11/10/2006' '00:00:00' 38]
 [970 'WARRIGAL_RD N of HIGH STREET_RD' '12/10/2006' '00:00:00' 28]
 [970 'WARRIGAL_RD N of HIGH STREET_RD' '13/10/2006' '00:00:00' 34]
 [970 'WARRIGAL_RD N of HIGH STREET_RD' '14/10/2006' '00:00:00' 61]
 [970 'WARRIGAL_RD N of HIGH STREET_RD' '15/10/2006' '00:00:00' 97]
 [970 'WARRIGAL_RD N of HIGH STREET_RD' '16/10/200

In [ ]:
# Filter the site
site_data = df[df['site_id'] == site_id].copy()
site_data = site_data.sort_values('timestamp')

# Only keep volume
volume_series = site_data['volume'].values.reshape(-1, 1)

# Normalize
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(volume_series)

# Create sequences
X, y = create_sequences(scaled_data, window_size)
X = X.reshape((X.shape[0], X.shape[1], 1))  # LSTM expects 3D input

# Split into train and test sets
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build LSTM model
model = Sequential([
    LSTM(64, activation='relu', input_shape=(window_size, 1)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

# Train model
model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[EarlyStopping(patience=5)]
)
